In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate Brat Reader

In [2]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [3]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [4]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read Brat annotations | load from pickles

In [5]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [6]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


## Define sampling function

In [7]:
rounds =10
seed= 14

In [8]:
len(train_docs), len(test_docs)

(303, 202)

## CRF Wrapper

In [9]:
from CRFWrapper_Sentence import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel


In [10]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Frequency', 'Duration', 'Route', 'Dosage', 'ADE', 'Drug', 'Strength', 'Reason', 'Form'}


In [11]:
crf_model=CRFModel(anno_types=annos)

## Convert training and testing docs into sentence level dataframe

In [12]:
_, train_df=convert_docs(train_docs, anno_types=annos)

In [13]:
_, test_df=convert_docs(test_docs, anno_types=annos)

In [14]:
train_df

,sentence_id,doc_name,token,label
0,0,100035.txt,Admission,O
1,0,100035.txt,Date,O
2,0,100035.txt,:,O
3,1,100035.txt,[,O
4,1,100035.txt,*,O
...,...,...,...,...
3151,929927,198406.txt,Followup,O
3152,929927,198406.txt,Instructions,O
3153,929927,198406.txt,:,O
3154,929927,198406.txt,\n,O


In [15]:
train_sentID_list = train_df['sentence_id'].to_list()
train_sentID_set = set(train_sentID_list)
train_sentID_uniqList = list(train_sentID_set)
test_sentID_list = test_df['sentence_id'].to_list()
test_sentID_set = set(test_sentID_list)
test_sentID_uniqList = list(test_sentID_set)

In [16]:
print(len(train_sentID_uniqList), len(test_sentID_uniqList))

51798 34334


In [18]:
int(1.0*len(train_sentID_uniqList)/20)

2589

## sampling simulator

In [19]:
from ALLSampler_Sentence import SamplingSimulator, ModelSamplingSimulator

In [20]:
m_simulator=ModelSamplingSimulator(total_sents=train_df, 
                                   total_round=10, 
                                   modelWrapper=crf_model, 
                                   eval_sents=test_df, 
                                   init_seed=seed)

## test run

In [20]:
scores=m_simulator.simulate_rounds() #sample sentences for 10 rounds; each round bootstrap sampling 200 for evaluation

2024-04-23 23:49:06.744 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 0.
2024-04-23 23:49:06.746 | DEBUG    | ALLSampler_Sentence:keep_sample:75 - The first round sampling will be random
2024-04-23 23:49:07.146 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 5179, remaining sentences: 46619
2024-04-23 23:49:09.124 | DEBUG    | CRFWrapper_Sentence:fit:284 - Reset and train CRF model...
2024-04-23 23:50:07.596 | DEBUG    | CRFWrapper_Sentence:fit:292 - Training complete.
2024-04-23 23:50:07.690 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:352 - Predicting eval sents...
2024-04-23 23:50:32.987 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:356 - Calculate scores from bootstrapping 200 times
2024-04-23 23:54:44.927 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:365 - complete
2024-04-23 23:54:45.055 | DEBUG    | ALLSampler_Sentence:simulate_rounds:103 - {'precision': 0.9460863337731906, 'recall': 0.77182582947748

In [21]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [22]:
pd.options.display.float_format='{:,.4f}'.format

In [23]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.9461,0.9429,0.9494,0.7718,0.7655,0.7785,0.8501,0.8460,0.8547
1,0.9419,0.9387,0.9452,0.8132,0.8063,0.8200,0.8728,0.8689,0.8770
2,0.9430,0.9401,0.9459,0.8264,0.8206,0.8327,0.8808,0.8774,0.8850
3,0.9446,0.9413,0.9476,0.8359,0.8306,0.8412,0.8870,0.8830,0.8907
4,0.9439,0.9405,0.9474,0.8453,0.8402,0.8507,0.8919,0.8883,0.8956
5,0.9456,0.9420,0.9485,0.8474,0.8415,0.8522,0.8938,0.8899,0.8973
6,0.9462,0.9432,0.9490,0.8494,0.8444,0.8544,0.8952,0.8915,0.8988
7,0.9455,0.9424,0.9485,0.8539,0.8486,0.8589,0.8974,0.8936,0.9011
8,0.9463,0.9431,0.9489,0.8573,0.8528,0.8622,0.8996,0.8962,0.9029
9,0.9459,0.9424,0.9489,0.8603,0.8557,0.8653,0.9010,0.8981,0.9043


## Bootstrap 3 different initials runs

In [21]:
boostrap_runs=3
total_round=10
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [22]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [23]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    m_simulator=ModelSamplingSimulator(total_sents=train_df, 
                                   total_round=10, 
                                   modelWrapper=crf_model, 
                                   eval_sents=test_df, 
                                   init_seed=seed)  
    scores=m_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-05-03 11:21:55.070 | INFO     | __main__:<module>:3 - start run 0.
2024-05-03 11:21:55.189 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 0.
2024-05-03 11:21:55.583 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 2589, remaining sentences: 49209
2024-05-03 11:27:10.196 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 1.
2024-05-03 11:28:14.181 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 5178, remaining sentences: 46620
2024-05-03 11:33:55.159 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 2.
2024-05-03 11:34:55.151 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 7767, remaining sentences: 44031
2024-05-03 11:41:09.971 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 3.
2024-05-03 11:42:06.511 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 10356, remaining sentences: 41442
2024-05

In [24]:
joblib.dump(all_scores, r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_5percent10round.joblib')


['../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_5percent10round.joblib']